In [1]:
import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_electricity as ml
import model_energy as me
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *
import auxiliary_definitions_transformations as adt

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(adt)


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2194: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2194: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'auxiliary_definitions_transformations' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/auxiliary_definitions_transformations.py'>

In [2]:


# initialize 
all_regions = None
dict_isos = {}
dict_replace_iso = sa.model_attributes.dict_attributes.get("region").field_maps.get("iso_alpha_3_to_region")
dict_sets = {}
field_country = "country"
field_iso = "iso_code3"
field_region = "nation"
field_year = "year"
fields_drop = [field_iso, field_region, field_year] # only apply later
field_time_period = sa.model_attributes.dim_time_period

# get some attributes
attr_region = sa.model_attributes.dict_attributes.get("region")
attr_sector = sa.model_attributes.dict_attributes.get("abbreviation_sector")
attr_strat = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_strategy_id}")
attr_time_period = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_time_period}")

# map each country to ISO code 3 and each code to 
dict_country_to_iso = dict((k, v.upper()) for k, v in attr_region.field_maps.get(f"{attr_region.key}_to_{attr_region.key}_abbreviation").items())
dict_iso_to_country = sf.reverse_dict(dict_country_to_iso)
dict_country_to_cf_region = attr_region.field_maps.get(f"{attr_region.key}_to_hourly_capacity_factor_region")
dict_iso_to_cf_region = dict((dict_country_to_iso.get(k), v) for k, v in dict_country_to_cf_region.items())

In [36]:
dict_fns_to_calib_note = {
    sa.fp_csv_climate_fields_by_country_simple: "Data are not specified for pre-2015, though can use 2015 values for previous years in calibration (can be treated as constant).",
    sa.fp_csv_soc_fields_by_country_simple: "Data are not specified for pre-2015, though can use 2015 values for previous years in calibration (can be treated as constant).",
    sa.fp_csv_nemomod_minimum_share_of_production_baselines: "Data need to be acquired for early calibration years, e.g., 2010-2014.",
    sa.fp_csv_nemomod_residual_capacity_inputs: "Data is based on empirical observations for all years, except for some very early years.",
    sa.fp_csv_nemomod_transmission_losses: "Data are based on most recent available. In some cases, obsrevations stop around 2014. Data are extended backward from earliest available time and forward from latest available time."
}

In [41]:
fields_ignore = [field_iso, field_country, field_time_period, field_year]

df_cols = []
for fp in dict_fns_to_calib_note.keys():
    
    desc = dict_fns_to_calib_note.get(fp)
    df_tmp = pd.read_csv(fp)
    
    fp_print = fns[0].split("lac_decarbonization")[1]
    df_cols += [(x, fp_print, desc) for x in df_tmp.columns if (x not in fields_ignore)] 
    
    
df_cols = pd.DataFrame(df_cols, columns = ["field", "path_in_lac_decarbonization", "calibration_note"])
df_cols.to_csv(
    sa.fp_csv_batch_data_field_index,
    index = None,
    encoding = "UTF-8"
)
